In [6]:
import numpy as np, imp, os, datetime as dt, pandas as pd, matplotlib.pyplot as plt

# Path to folder containing wrapper modules
wrapper_fpath = (r"..\inca.py")
optimize_funs_fpath = (r'..\inca_calibration.py')

wr = imp.load_source('inca',wrapper_fpath)
cf = imp.load_source('inca_calibration', optimize_funs_fpath)

In [17]:
catchment = 'Morsa' # Choose from: 'Morsa','Tarland'
wr.initialize('simplyp.dll')
dataset = wr.DataSet.setup_from_parameter_and_input_files('../../Applications/SimplyP/%s/%sParameters.dat'%(catchment,catchment),
                                                            '../../Applications/SimplyP/%s/%sInputs.dat'%(catchment,catchment))

In [18]:
print ('Index sets: %s' %dataset.get_index_sets())
print ('Indices in 1st index set: %s' %dataset.get_indexes(dataset.get_index_sets()[0]))
print ('Indices in 2nd index set: %s' %dataset.get_indexes(dataset.get_index_sets()[1]))

Index sets: ['Reaches', 'Landscape units']
Indices in 1st index set: ['Kure']
Indices in 2nd index set: ['Arable', 'Improved grassland', 'Semi-natural']


In [19]:
paramTupleList = dataset.get_parameter_list()
paramList = [i[0] for i in paramTupleList]
print (paramList)

['Timesteps', 'Start date', 'Latitude', 'Initial snow depth as water equivalent', 'Degree-day factor for snowmelt', 'Proportion of precipitation that contributes to quick flow', 'PET multiplication factor', 'Soil field capacity', 'Baseflow index', 'Groundwater time constant', 'Minimum groundwater flow', 'Gradient of reach velocity-discharge relationship', 'Exponent of reach velocity-discharge relationship', 'Catchment area', 'Reach length', 'Initial in-stream flow', 'Soil water time constant', 'Land use proportions', 'Dynamic erodibility', 'Reach sediment input scaling factor', 'Sediment input non-linear coefficient', 'Day of year when soil erodibility is at its max for spring-grown crops', 'Day of year when soil erodibility is at its max for autumn-grown crops', 'Reach slope', 'Proportion of spring grown crops', 'Mean slope of land in the subcatchment', 'Vegetation cover factor', 'Reduction of load in sediment', 'Soil mass per m2', 'Phosphorous sorption coefficient', 'Net annual P inp

In [24]:
# Drop non-variable params
constantParams = ['Timesteps','Start date', 'Latitude','Catchment area', 'Reach length', 'Land use proportions',
                  'Land use proportions from newly-converted', 'Reach slope', 'Dynamic EPC0', 'Is input to water body',
                  'Proportion of spring grown crops', 'Mean slope of land in the subcatchment',
                  'Reduction of load in sediment', 'Soil mass per m2', 'Phosphorous sorption coefficient',
                  'Net annual P input to newly-converted soil', 
                  'Dynamic erodibility', 'Newly-converted type', 'Initial total soil P content',
                  'Net annual P input to agricultural soil',
                  'Exponent of stream velocity-discharge relationship',
                  'Day of year when soil erodibility is at its max for spring-grown crops',
                  'Day of year when soil erodibility is at its max for autumn-grown crops',
                  'Vegetation cover factor']
initialParams = ['Initial snow depth','Initial in-stream flow', 'Initial soil temperature']
paramList = [x for x in paramList if x not in constantParams]
paramList = [x for x in paramList if x not in initialParams]
paramList

['Degree-day factor for snowmelt',
 'Proportion of precipitation that contributes to quick flow',
 'PET reduction factor',
 'Soil field capacity',
 'Baseflow index',
 'Groundwater time constant',
 'Minimum groundwater flow',
 'Gradient of stream velocity-discharge relationship',
 'Soil water time constant',
 'Reach sediment input scaling factor',
 'Sediment input non-linear coefficient',
 'Groundwater TDP concentration',
 'Particulate P enrichment factor',
 'SRP fraction',
 'Reach effluent TDP inputs',
 'Initial soil water EPC0']

In [7]:
# Make a dataframe for params to go in and write to csv
param_df = pd.DataFrame(data=paramList, index=np.arange(len(paramList)), columns=['Param_long'])
fpath = r'C:\Data\GitHub\INCABuilder\PythonWrapper\SimplyP\SimplyP_calParams_ranges.csv'
param_df.to_csv(fpath)

In [25]:
# Which of these params are indexed, and what are the indices?
# Add a row in the csv written out above for each param which is indexed
for param in paramList:
    indexers = dataset.get_parameter_index_sets(param)
    if len(dataset.get_parameter_index_sets(param))>=1:
        indices = dataset.get_indexes(indexers[0])
        print ('%s: %s' %(param, indices))

Soil water time constant: ['Arable', 'Improved grassland', 'Semi-natural']
Reach effluent TDP inputs: ['Tarland1']
Initial soil water EPC0: ['Arable', 'Improved grassland', 'Semi-natural']


In [22]:
# Read in amended csv with short names and param ranges
fpath = r'C:\Data\GitHub\INCABuilder\PythonWrapper\SimplyP\SimplyP_calParams_ranges.csv'
param_df = pd.read_csv(fpath)
param_df['calibration'] = list(zip(param_df.Param_long, param_df.Indexes)) #NB need to convert string to list for indexes

# Extract inputs
param_min = param_df['Min']
param_max = param_df['Max']
labels_short = param_df['ShortName']
calibration = list(param_df['calibration'].values)